In [33]:
!pip install spiceypy
!pip install skyfield

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install wget
import wget
wget.download('https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls', 'naif0012.tls')
wget.download('https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_latest_high_prec.bpc', 'earth_latest_high_prec.bpc')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'earth_latest_high_prec (2).bpc'

In [19]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de431.tpc


--2023-05-13 01:13:42--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de431.tpc
Resolving naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 137.78.232.95
Connecting to naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6291 (6.1K)
Saving to: ‘gm_de431.tpc’

gm_de431.tpc        100%[===================>]   6.14K  --.-KB/s    in 0s      

2023-05-13 01:13:43 (139 MB/s) - ‘gm_de431.tpc’ saved [6291/6291]



In [34]:
import numpy as np
import spiceypy as spice
from scipy.optimize import least_squares
from skyfield.api import Topos, load

# Load SPICE kernels
spice.furnsh('naif0012.tls')
spice.furnsh('pck00010.tpc')
spice.furnsh('gm_de431.tpc')



# Constants
R2D = 180.0 / np.pi
D2R = np.pi / 180.0
MJD_OFFSET = 2400000.5


#Question 1

# Table1 data
data = np.array([
    [265.3869518, 38.4539569, 19.3547079, 55.5481428, 60055.1340277],
    [286.0157158, 47.7809854, 14.2703198, 38.2354388, 60055.1354166],
    [306.8358871, 50.4514381, 8.1356679, 25.9457472, 60055.1368055],
    [113.1072289, 76.1961378, 40.4709053, 341.8114490, 60055.2104166],
    [44.4521793, 71.6883545, 23.6357032, 350.7827597, 60055.2118055],
    [27.5410256, 61.8733178, 12.4517325, 355.0101055, 60055.2131944],
    [128.5555539, 21.4050426, 10.1299902, 289.4238109, 60055.2854166],
    [116.9894450, 32.8319850, 9.4168870, 305.2967884, 60055.2868055],
    [102.4669647, 41.0669857, 6.8773277, 319.7464073, 60055.2881944]
])

# Champaign, Illinois coordinates
lat = 40.1164
lon = -88.2434
elevation = 233

# Functions
def ecef2eci(pos_ecef, et):
    ecef2eci_rot = spice.pxform('IAU_EARTH', 'J2000', et)
    return np.dot(ecef2eci_rot, pos_ecef)


def topocentric_to_ecef(az, el, slant_range, lat, lon, alt):
    # Convert to radians
    az_rad = az * D2R
    el_rad = el * D2R
    lat_rad = lat * D2R
    lon_rad = lon * D2R

    # Calculate ECEF position
    radii = spice.bodvrd('EARTH', 'RADII', 3)[1]
    a = radii[0]  # semi-major axis
    b = radii[2]  # semi-minor axis
    f = (a - b) / a  # Earth's flattening factor

    N = a / np.sqrt(1 - (2 * f) + (f ** 2) * (np.sin(lat_rad) ** 2))
    x = (N + alt) * np.cos(lat_rad) * np.cos(lon_rad) + slant_range * np.cos(el_rad) * np.cos(az_rad)
    y = (N + alt) * np.cos(lat_rad) * np.sin(lon_rad) + slant_range * np.cos(el_rad) * np.sin(az_rad)
    z = (N * (1 - f) ** 2 + alt) * np.sin(lat_rad) + slant_range * np.sin(el_rad)

    return np.array([x, y, z])


def residuals(params, data):
    # Unpack parameters
    x0, y0, z0, vx0, vy0, vz0 = params

    # Calculate residuals
    res = []
    for row in data:
        ra, dec, alt, az, mjd = row
        et = (mjd - MJD_OFFSET) * 86400.0
        pos_ecef = topocentric_to_ecef(az, alt, elevation, lat, lon, 0)
        pos_eci = ecef2eci(pos_ecef, et)
        pos_sat = np.array([x0 + vx0 * et, y0 + vy0 * et, z0 + vz0 * et])
        delta_pos = pos_sat - pos_eci
        delta_ra = np.arctan2(delta_pos[1], delta_pos[0]) * R2D - ra
        delta_dec = np.arctan2(delta_pos[2], np.sqrt(delta_pos[0] ** 2 + delta_pos[1] ** 2)) * R2D - dec
        res.extend([delta_ra, delta_dec])

    return res

# Initial guess
x0 = [7000, 0, 0, 0, 7.5, 0]

# Least squares optimization
result = least_squares(residuals, x0, args=(data,), verbose=2)

# Print results
print("\n\nBest fit initial state vector (ECI):")
print("Position (km):", result.x[0:3])
print("Velocity (km/s):", result.x[3:])

eci_position = result.x[0:3]
eci_velocity = result.x[3:]
average_mjd = np.mean(data[:, 4])
epoch_et = (average_mjd - MJD_OFFSET) * 86400.0
mu = spice.bodvrd('EARTH', 'GM', 1)[1][0]  # Earth's gravitational constant
state_vector = np.hstack((eci_position, eci_velocity))
orbital_elements = spice.oscelt(state_vector, epoch_et, mu)

print("\n\nGeocentric orbital elements:")
print("Semi-major axis (km):", orbital_elements[0])
print("Eccentricity:", orbital_elements[1])
print("Inclination (deg):", np.degrees(orbital_elements[2]))
print("Longitude of the ascending node (deg):", np.degrees(orbital_elements[3]))
print("Argument of periapsis (deg):", np.degrees(orbital_elements[4]))
print("True anomaly (deg):", np.degrees(orbital_elements[5]))
print("Epoch (ET):", epoch_et)


#Question 2

stations_url = 'https://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
def orbital_elements_difference(satellite, computed_elements):
    sat_elements = [
        satellite.model.a * (1 - satellite.model.ecco),  # Semi-major axis
        satellite.model.ecco,                            # Eccentricity
        satellite.model.inclo,                           # Inclination
        satellite.model.nodeo,                           # Longitude of the ascending node
        satellite.model.argpo,                           # Argument of periapsis
        satellite.model.mo,                              # Mean anomaly
    ]

    # Calculate the difference between the computed and TLE orbital elements
    difference = sum(abs(np.array(sat_elements) - np.array(computed_elements)))

    return difference
computed_elements = [
    orbital_elements[0],
    orbital_elements[1],
    orbital_elements[2],
    orbital_elements[3],
    orbital_elements[4],
    orbital_elements[5],
]

min_difference = float('inf')
best_match = None

for sat in satellites:
    difference = orbital_elements_difference(sat, computed_elements)
    if difference < min_difference:
        min_difference = difference
        best_match = sat

print("\n\nBest match satellite:")
print("Name:", best_match.name)
print("Object ID:", best_match.model.satnum)
print("NORAD CAT ID:", best_match.model.satnum)


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.2514e+05                                    1.68e+04    
       1              2         3.2454e+05      5.99e+02       7.00e+03       1.80e+01    
       2              3         2.9396e+05      3.06e+04       1.75e+03       1.60e+01    
       3              4         2.4205e+05      5.19e+04       3.50e+03       1.00e+01    
       4              5         1.7137e+05      7.07e+04       7.00e+03       8.10e+00    
       5              6         1.0293e+05      6.84e+04       1.40e+04       2.59e+00    
       6              7         5.8087e+04      4.48e+04       2.80e+04       7.13e-01    
       7              9         4.8779e+04      9.31e+03       1.40e+04       3.58e-01    
       8             10         4.5604e+04      3.18e+03       2.80e+04       9.55e-02    
       9             11         4.5189e+04      4.15e+02       5.60e+04       2.42e-02    